In [2]:
!pip install plotly

  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
Using cached plotly-5.24.1-py3-none-any.whl (19.1 MB)
Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)


In [3]:
from wine_classification_hyperparameter_search_v2 import (
    get_dataframe,
    create_search_grid,
    train_model,
    compare_model_results,
    analyze_model
)

%load_ext autoreload
%autoreload 2

16:50:50.483547 WARNING  type_engine.py:1358 - Unsupported Type typing.Any      
                         found, Flyte will default to use PickleFile as the     
                         transport. Pickle can only be used to send objects     
                         between the exact same version of Python, and we       
                         strongly recommend to use python type that flyte       
                         support.

In [4]:
grid = {
    'max_depth': [10, 50, 100],
    'max_features': [None, 'sqrt'],
    'n_estimators': [100, 2000]
}

In [5]:
dataframe = get_dataframe()
dataframe.head()

╭─────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────╮
│ /Users/tchase/Documents/repos/flyte_union_public/new_folder/my_new_env/lib/python3.11/site-packages/flytekit/co │
│ re/promise.py:623 in create_native_named_tuple                                                                  │
│                                                                                                                 │
│ ❱  623 │   │   │   return TypeEngine.to_python_value(ctx, promises.val, v)                                      │
│                                                                                                                 │
│ /Users/tchase/Documents/repos/flyte_union_public/new_folder/my_new_env/lib/python3.11/site-packages/flytekit/co │
│ re/type_engine.py:1101 in to_python_value                                                                       │
│                                                                                                                 │
│ ❱ 1101 │   │   return transformer.to_python_value(ctx, lv, expected_python_type)                                │
│                                                                                                                 │
│ /Users/tchase/Documents/repos/flyte_union_public/new_folder/my_new_env/lib/python3.11/site-packages/flytekit/ty │
│ pes/structured/structured_dataset.py:830 in to_python_value                                                     │
│                                                                                                                 │
│ ❱ 830 │   │   return self.open_as(ctx, lv.scalar.structured_dataset, df_type=expected_python_t                  │
│                                                                                                                 │
│ /Users/tchase/Documents/repos/flyte_union_public/new_folder/my_new_env/lib/python3.11/site-packages/flytekit/ty │
│ pes/structured/structured_dataset.py:865 in open_as                                                             │
│                                                                                                                 │
│ ❱ 865 │   │   result = decoder.decode(ctx, sd, updated_metadata)                                                │
│                                                                                                                 │
│ /Users/tchase/Documents/repos/flyte_union_public/new_folder/my_new_env/lib/python3.11/site-packages/flytekit/ty │
│ pes/structured/basic_dfs.py:137 in decode                                                                       │
│                                                                                                                 │
│ ❱ 137 │   │   │   return pd.read_parquet(uri, columns=columns, storage_options=kwargs)                          │
│                                                                                                                 │
│ /Users/tchase/Documents/repos/flyte_union_public/new_folder/my_new_env/lib/python3.11/site-packages/pandas/io/p │
│ arquet.py:667 in read_parquet                                                                                   │
│                                                                                                                 │
│ ❱ 667 │   return impl.read(                                                                                     │
│                                                                                                                 │
│ /Users/tchase/Documents/repos/flyte_union_public/new_folder/my_new_env/lib/python3.11/site-packages/pandas/io/p │
│ arquet.py:267 in read                                                                                           │
│                                                                                                                 │
│ ❱ 267 │   │   path_or_handle, handles, filesystem = _g

In [5]:
hyperparameters = create_search_grid(grid = grid)
hyperparameters

[{'max_depth': 10, 'max_features': None, 'n_estimators': 100},
 {'max_depth': 10, 'max_features': None, 'n_estimators': 2000},
 {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100},
 {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 2000},
 {'max_depth': 50, 'max_features': None, 'n_estimators': 100},
 {'max_depth': 50, 'max_features': None, 'n_estimators': 2000},
 {'max_depth': 50, 'max_features': 'sqrt', 'n_estimators': 100},
 {'max_depth': 50, 'max_features': 'sqrt', 'n_estimators': 2000},
 {'max_depth': 100, 'max_features': None, 'n_estimators': 100},
 {'max_depth': 100, 'max_features': None, 'n_estimators': 2000},
 {'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 100},
 {'max_depth': 100, 'max_features': 'sqrt', 'n_estimators': 2000}]

In [6]:
import pandas as pd
pd.DataFrame(hyperparameters)

,max_depth,max_features,n_estimators
0,10,None,100
1,10,None,2000
2,10,sqrt,100
3,10,sqrt,2000
4,50,None,100
5,50,None,2000
6,50,sqrt,100
7,50,sqrt,2000
8,100,None,100
9,100,None,2000


In [7]:
hyperparameters[0].keys()

dict_keys(['max_depth', 'max_features', 'n_estimators'])

In [ ]:
from functools import partial
from flytekit import map_task

models = map_task(partial(train_model, dataframe=dataframe))(hyperparameters=hyperparameters)
print(len(models))

In [ ]:
best_model = compare_model_results(models = models, hyperparameters=hyperparameters, dataframe = dataframe, force_plot=True)

In [ ]:
analyze_model(best_model, dataframe = dataframe, force_plot=True)